# 5. Benchmark `deeplake`

In [1]:
import deeplake
import os
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import IPython.display as ipd
from utils import time_me, time_me_seconds, SAMPLE_RATE
from benchmarks import run_all_benchmarks, run_cpu_benchmark

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./secret/hpml-399816-55d1dc19c012.json"

In [3]:
def deeplake_cropper(crop_duration):
    crop_samples = int(SAMPLE_RATE * crop_duration)

    def crop(audio):
        audio = audio[:, 0]
        audio = audio / np.abs(audio).max()  # peak normalize
        audio = audio.astype(np.float32)  # convert to f32
        num_samples = audio.shape[0]
        assert num_samples != 0
        if num_samples < crop_samples:
            # pad if input is short
            audio = np.pad(audio, (0, crop_samples - num_samples))
        elif num_samples > crop_samples:
            # crop if it is too long
            rand_start = np.random.randint(0, num_samples - crop_samples)
            audio = audio[rand_start : rand_start + crop_samples]
        else:
            # just right :)
            pass
        assert audio.shape[0] == crop_samples

        return audio[None, :]

    return crop

In [5]:
@time_me_seconds
def build_deeplake_dataloader(
    deeplake_path,
    crop_duration=3.0,
    batch_size=32,
    shuffle_buffer=2048,
    num_workers=4,
    pin_memory=True,
):
    shuffle_buffer_size = (
        310 / 2048 * shuffle_buffer
    )  # deeplake uses MB as measurement for buffer
    ds = deeplake.load(deeplake_path)
    dataloader = ds.pytorch(
        tensors=["audio", "accent", "gender", "age"],
        transform={
            "audio": deeplake_cropper(crop_duration=crop_duration),
            "accent": None,
            "gender": None,
            "age": None,
        },
        num_workers=num_workers,
        batch_size=batch_size,
        pin_memory=pin_memory,
        shuffle=True,
        buffer_size=shuffle_buffer_size,
    )
    single_worker_dataloader = ds.pytorch(
        tensors=["audio", "accent", "gender", "age"],
        transform={
            "audio": deeplake_cropper(crop_duration=crop_duration),
            "accent": None,
            "gender": None,
            "age": None,
        },
        num_workers=0,
        batch_size=batch_size,
        pin_memory=pin_memory,
        shuffle=True,
        buffer_size=shuffle_buffer_size,
    )
    return dataloader, single_worker_dataloader

In [6]:
local_deeplake_path = "./data/deeplake/"
cloud_deeplake_path = "gcs://hpml-project/deeplake/"

In [7]:
tag = "deeplake_local"
dataloader, single_worker_dataloader = build_deeplake_dataloader(local_deeplake_path)
run_all_benchmarks(dataloader, single_worker_dataloader, tag=tag)


[2023-12-17_18-45-34]	
+-----------------------------------+
|Beginning build_deeplake_dataloader|
+-----------------------------------+
./data/deeplake/ loaded successfully.



[2023-12-17_18-45-39]	
+-------------------------------------------+
|build_deeplake_dataloader took 5.92 seconds|
+-------------------------------------------+



STAGE:2023-12-17 18:45:40 3422188:3422188 ActivityProfilerController.cpp:311] Completed Stage: Warm Up


Number of parameters of model: 64,628,259


  0%|                                                                                             | 0/200 [00:00<?, ?it/s]
Please wait, filling up the shuffle buffer with samples.:   0%|                                | 0.00/296M [00:00<?, ?B/s]
Please wait, filling up the shuffle buffer with samples.:   0%|                      | 188k/296M [00:03<1:32:55, 55.6kB/s]
Please wait, filling up the shuffle buffer with samples.:   4%|▉                      | 11.9M/296M [00:03<01:00, 4.91MB/s]
Please wait, filling up the shuffle buffer with samples.:   8%|█▊                     | 23.6M/296M [00:03<00:25, 11.1MB/s]
Please wait, filling up the shuffle buffer with samples.:  12%|██▋                    | 35.3M/296M [00:03<00:14, 18.6MB/s]
Please wait, filling up the shuffle buffer with samples.:  16%|███▋                   | 47.1M/296M [00:03<00:09, 27.0MB/s]
Please wait, filling up the shuffle buffer with samples.:  20%|████▌                  | 58.8M/296M [00:04<00:06, 37.2MB/s]
Please wait, fil

Shuffle buffer filling is complete.


/home/kaandonbekci/miniconda3/envs/ccu/lib/python3.11/site-packages/torch/nn/modules/conv.py:309: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
100%|███████████████████████████████████████████████████████████████████████████████████| 200/200 [00:17<00:00, 11.13it/s]
STAGE:2023-12-17 18:45:58 3422188:3422188 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-12-17 18:45:58 3422188:3422188 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


Time to 1st batch: 6.55 seconds


/tmp/ipykernel_3422188/1012810662.py:6: RuntimeWarning: invalid value encountered in divide
  audio = audio / np.abs(audio).max()  # peak normalize


In [8]:
tag = "deeplake_cloud"
dataloader, single_worker_dataloader = build_deeplake_dataloader(cloud_deeplake_path)
run_all_benchmarks(dataloader, single_worker_dataloader, tag=tag)


[2023-12-17_18-47-25]	
+-----------------------------------+
|Beginning build_deeplake_dataloader|
+-----------------------------------+


/

gcs://hpml-project/deeplake/ loaded successfully.



[2023-12-17_18-48-09]	
+--------------------------------------------+
|build_deeplake_dataloader took 44.42 seconds|
+--------------------------------------------+



STAGE:2023-12-17 18:48:09 3422188:3422188 ActivityProfilerController.cpp:311] Completed Stage: Warm Up


Number of parameters of model: 64,628,259


  0%|                                                                                             | 0/200 [00:00<?, ?it/s]
Please wait, filling up the shuffle buffer with samples.:   0%|                                | 0.00/296M [00:00<?, ?B/s]
Please wait, filling up the shuffle buffer with samples.:   0%|                      | 188k/296M [00:11<4:58:55, 17.3kB/s]
Please wait, filling up the shuffle buffer with samples.:   4%|▉                      | 11.9M/296M [00:12<03:38, 1.36MB/s]
Please wait, filling up the shuffle buffer with samples.:  12%|██▋                    | 35.3M/296M [00:13<01:02, 4.38MB/s]
Please wait, filling up the shuffle buffer with samples.:  16%|███▋                   | 47.1M/296M [00:14<00:47, 5.45MB/s]
Please wait, filling up the shuffle buffer with samples.:  24%|█████▍                 | 70.5M/296M [00:16<00:27, 8.48MB/s]
Please wait, filling up the shuffle buffer with samples.:  32%|███████▎               | 93.9M/296M [00:17<00:19, 11.0MB/s]
Please wait, fil

Shuffle buffer filling is complete.


 38%|████████████████████████████████▎                                                   | 77/200 [00:55<01:27,  1.40it/s]
STAGE:2023-12-17 18:49:05 3422188:3422188 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-12-17 18:49:05 3422188:3422188 ActivityProfilerController.cpp:321] Completed Stage: Post Processing

KeyboardInterrupt

